In [ ]:
!pip install --upgrade ibm-watson

In [ ]:
# Paste your API key for IBM Watson Visual Recognition below:
my_apikey = 'your api key'

In [ ]:
#create an instance
from ibm_watson import VisualRecognitionV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
authenticator = IAMAuthenticator(my_apikey)

visrec = VisualRecognitionV3('2018-03-19', 
                             authenticator=authenticator)

In [ ]:
#we will create a classifier for dogs, so download the zip files that we need for train the classifier
import urllib.request

# Downloading Beagle dataset
urllib.request.urlretrieve("http://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/Beagle.zip", 
                           "beagle.zip")

# Downloading Husky dataset
urllib.request.urlretrieve("http://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/Husky.zip", 
                           "husky.zip")

# Downloading Golden Retriever dataset
urllib.request.urlretrieve("http://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/GoldenRetriever.zip", 
                           "goldenretriever.zip") #note that we should remove any hyphens from the zip file name

In [ ]:
#Lets train our Visual Recognition model to recognize the three breeds of dogs using the create_classifier method from the Watson Image Recognition API
import json
with open('beagle.zip', 'rb') as beagle, \
     open('goldenretriever.zip', 'rb') as gretriever, \
     open('husky.zip', 'rb') as husky:
        response = visrec.create_classifier(name="dogbreedclassifier",
                                        positive_examples={'beagle': beagle, \
                                                           'goldenretriever': gretriever, \
                                                           'husky': husky})
print(json.dumps(response.get_result(), indent=2))

In [ ]:
#lets grab the classifier id
classifier_id = response.get_result()["classifier_id"]
classifier_id

In [ ]:
#Depending on the number of images, it may take a Several Minutes for Watson to build a custom classifier. Please wait tell you get Good to Go
Status = visrec.get_classifier(classifier_id=classifier_id, verbose=True).get_result()['status']
if Status=='training': 
    print ("Please, Wait to complete training.......")
else:
    print("Good to go ")

In [ ]:
#List all your (custom) classifiers in IBM Watson
visrec.list_classifiers(verbose=True).get_result()

In [ ]:
#testing it
from pandas.io.json import json_normalize

def getdf_visrec(url, classifier_ids, apikey = my_apikey):
    
    json_result = visrec.classify(url=url,
                              threshold='0.6',
                              classifier_ids=classifier_id).get_result()
    
    json_classes = json_result['images'][0]['classifiers'][0]['classes']
    
    df = json_normalize(json_classes).sort_values('score', ascending=False).reset_index(drop=True)
    
    return df

In [ ]:
getdf_visrec(url = 'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/GoldenRetriever1_stacked.jpg',
            classifier_ids="dogbreedclassifier_279527932")
